**Question 1. **You are given a train data set having 1000 columns and 1 million rows. The data set is based on a classification problem. Your manager has asked you to reduce the dimension of this data so that model computation time can be reduced. Your machine has memory constraints. What would you do? (You are free to make practical assumptions.)


**Answer 1 **

From the question, it is not clear, whether data is a) numeric b) textual or c) image. Although, it looks to be numeric data. Dimensional reduction techniques varies for each category of data. For numeric data, we can use any of the following:

1.Principal Component Analysis - PCA


procedure that transforms a number of correlated variables into a (smaller) number of uncorrelated variables called principal components. What happens if PCA does not find any correlated variables? 

2. Latent Symantic Analysis - LSA

Latent Semantic Analysis (LSA) is a prominent dimensionality reduction technique. It is an application of truncated singular value decomposition (SVD) and so uses only techniques from linear algebra (no machine learning needed).

3. Latent Drichlet Allocation - LDA

It is kind of LSA

4. Autoencoders

This is a neural net technique. An autoencoder is a machine learning model that seeks to learn parameters that predict its own input. This is meaningful when there are intermediate representations that have lower dimensionality than the inputs. These provide a reduced-dimensional view of the data akin to those learned by LSA, but now we have a lot more design choices and a lot more potential to learn higher-order associations in the underyling data.

5. L1 regularization. 

This makes the unimportant feature weights to zero, thus reducing number of features.

Also. I believe, Dimensional reduction depends on the algorithm under consideration. for example, in case of NeuralNet, we can simply use dropout. t-SNE is another dimensionality reduction technique for data visualization.


**Question 2. **
You are given a data set on cancer detection. You’ve build a classification model and achieved an accuracy of 96%. Why shouldn’t you be happy with your model performance? What can you do about it?


**Answer 2. **

Beacuse this is life critical application and therefore, precision should be 100% or more. Going thru the misclassified samples and analyze it, what features can be added to improve performance. You may have to change the algorthm also. 


**Question 3.**
You came to know that your model is suffering from low bias and high variance. Which algorithm should you use to tackle it? Why?



**Answer 3. **
low bias and high variance model means model is overfitted. Meaning it is not generize enough to predict an unseen data sample. It is fitted with training data only, it does not understand anything outside of training data. To overcome, we can use L2 regularzation. Try other algorithms if the project allowed. For examle, Decision Trees, are generally prone to overfitting. So, you can change to SVM or logistic regression.In case of Neural Net, we use a techniue known as dropout to control overfitting. 



# Supermarket Optimization
 
You’ve been hired as a market consultant to try and help a local supermarket come up with better placement of items based on buyer’s preferences, and towards that goal you’d like to identify certain association rules based on existing records of buyer’s transactions.
 
You are given as input:

• A transaction database - a file consisting of a single row per transaction, with individual product's SKUs given as space–separated integers. A single transaction consisting of products with SKUs 1001, 1002 and 1003 would have a line that looks like: ‘1001 1002 1003' 

• A minimal ’support level’ parameter, sigma – a positive integer

Implement, in Python or Java, an efficient algorithm for generating all frequent item sets of size 3 or more: groups of 3 or more items that appear together in the transactions log at least as often as the support level parameter value. For example, given a value of sigma = 2, all sets of 3 items that appear 2 or more times together in the transaction log should be returned.
 
The results should be returned as a file with the following format:
``` 
<item set size (N)>, <co-occurrence frequency>, <item 1 id >, <item 2 id>, …. <item N id>
```
 
Run the algorithm on the attached transaction log file and provide the results obtained for a value of sigma = 4



In [1]:
import pandas as pd
import numpy as np
import sys
from collections import Counter
import csv
from itertools import combinations

Load order data

In [2]:
supermarket_data = pd.read_csv("retail_25k.dat", header=None)

In [3]:
#below file created for testing only
#supermarket_data = pd.read_csv("test.data", header=None)

In [4]:
pd.set_option('max_colwidth', 800)
supermarket_data.insert(0, 'tran_id', supermarket_data.index, allow_duplicates = False)
supermarket_data = supermarket_data.rename(columns={supermarket_data.columns[1]: "item_id"})
supermarket_data.head(15)

,tran_id,item_id
0,0,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29
1,1,30 31 32
2,2,33 34 35
3,3,36 37 38 39 40 41 42 43 44 45 46
4,4,38 39 47 48
5,5,38 39 48 49 50 51 52 53 54 55 56 57 58
6,6,32 41 59 60 61 62
7,7,3 39 48
8,8,63 64 65 66 67 68
9,9,32 69


In [5]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))
print('transactions -- dimensions: {0};   size: {1}'.format(supermarket_data.shape, size(supermarket_data)))

transactions -- dimensions: (25000, 2);   size: 2.75 MB


In [6]:
#create a new column item_set_count
supermarket_data['item_set_count'] = supermarket_data['item_id'].apply(lambda x : len(x.split()))

In [7]:
supermarket_data.head(5)

,tran_id,item_id,item_set_count
0,0,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29,30
1,1,30 31 32,3
2,2,33 34 35,3
3,3,36 37 38 39 40 41 42 43 44 45 46,11
4,4,38 39 47 48,4


In [8]:
def remove_item_set(df, min_itemset_size=3):
    df.drop(df.loc[df['item_set_count'] < min_itemset_size].index, inplace=True)
    return df

removed_supermarket_data = remove_item_set(supermarket_data)
print(removed_supermarket_data.shape)
removed_supermarket_data.head(15)

(22516, 3)


,tran_id,item_id,item_set_count
0,0,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29,30
1,1,30 31 32,3
2,2,33 34 35,3
3,3,36 37 38 39 40 41 42 43 44 45 46,11
4,4,38 39 47 48,4
5,5,38 39 48 49 50 51 52 53 54 55 56 57 58,13
6,6,32 41 59 60 61 62,6
7,7,3 39 48,3
8,8,63 64 65 66 67 68,6
10,10,48 70 71 72,4


In [9]:
# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item, min_item_set_size):
    item_list = order_item.split()
#    print(item_list)
    result = []
    for item_pair in combinations(item_list, min_item_set_size):
        result.append(item_pair)
    return result


In [10]:
def write_output(dict_file):
    csv_columns = ['item set size (N)','co-occurrence frequency','item_ids']
    with open('results.csv', 'a') as f:  
        w = csv.DictWriter(f, fieldnames=csv_columns)
        w.writeheader()
        w.writerow(dict_file)


In [11]:
def supermarket_optimization(df, sigma = 4, min_item_set_size = 3):
    
    tokenized_items = df['item_id'].apply(lambda x : get_item_pairs(x, min_item_set_size))
#    print ( tokenized_items )
    remove_list = []
    cnt = Counter()
    for items in tokenized_items:
        for item in items:
            cnt[item] += 1
#min support            
    result_dict = {k:cnt[k] for k in cnt if cnt[k] > sigma}
#    print(result_dict)
    return result_dict


In [ ]:
i=3
sigma = 4
with open('supermarket_optimization_results.csv', 'w') as writeFile:
    while True: 
        file_list = []
        out = supermarket_optimization(removed_supermarket_data, sigma, i)
        i += 1 
        if i > 10:
            break;

#out_dict = {'item set size (N)': 0,'co-occurrence frequency': 0, 'item_ids':0 }
        for key in out:
            file_list.append([len(key), out[key], key])
        writer = csv.writer(writeFile)
        writer.writerows(file_list)
